![Cradlepoint](logo.svg)

---

# **PART 2** - NCOS API Deep Dive

#### This section will cover:

### Connection Manager

- #### How **WAN Rules** (Profiles) become **Devices** and how to get their **status**.
- #### Identifier Fields: "UID" vs "\_id\_"

### Additional CSClient() methods:
- #### .post()
- #### .delete()
- #### .decrypt()
- #### .patch()

### Using the `control/` tree

---

## First, we need to import the **cslient** and **json** modules and instantiate the **CSClient()** class:

> Each notebook has a separate kernel and memory space so we have to start over.

---
```python
import cslient
import json
cp = cslient.CSClient('SDK Lab')
```
---

#### Your turn:

# Connection Manager

## WAN Rules (Profiles):

### The Connection Manager lists "WAN Device Interface Profiles".  

#### These profiles, or rules, are found in `config/wan/rules2`.  For simplicity we will just call them **WAN rules**.  

#### WAN rules are used to configure WAN devices.

> #### **PROTIP:** Don't forget the "2" on the end of `config/wan/rules2`!  This signifies the 2nd version of rules.  There is an old path `config/wan/rules` - do not use it!

<br>

### **Get** and **pretty-print** `config/wan/rules2`:

## `config/wan/rules2/` is a **list** of WAN rules, or profiles.

### The fields of a WAN rule:


## **"\_id\_"**

#### "\_id\_" fields are defined as type **uuid** in the NCOS DTD.

> #### UUID format: In its canonical textual representation, the 16 octets of a UUID are represented as 32 hexadecimal (base-16) digits, displayed in five groups separated by hyphens, in the form 8-4-4-4-12 for a total of 36 characters (32 hexadecimal characters and 4 hyphens). For example:

`123e4567-e89b-12d3-a456-4266141740001`

#### NCOS generates an "\_id\_" for you when you create a new list item with **cp.post()**

> Don't create your own uuids!

#### "\_id\_" fields are used in many lists to identify or refer to items.  Other features will also use these to refer to an item in a list.  

#### For example, the "Ethernet" profile name is known to NCOS as "00000000-a81d-3590-93ca-8b1fcfeb8e14" (or similar).

> NCM uses these identifiers to elegantly correlate layers when merging group and device configs!

#### They can be also be used **directly in place of list indexes** in paths!

#### For example, this path:

`config/wan/rules2/0`

#### Is the same as:

`config/wan/rules2/00000000-a81d-3590-93ca-8b1fcfeb8e14`


## **"trigger_name"**

#### The **trigger_name** is simply the name for the WAN rule (Profile) that is displayed in the Connection Manager.
> The **trigger_name** has no effect on the function of the WAN interface.

## **"trigger_string"**

#### A **trigger_string** is a system for matching WAN devices to a corresponding rule (profile).
#### It is a series of criteria sets separated by pipes (|)
#### Example: `"type|is|mdm%tech|is|lte/3g"`

#### Each of those criteria sets matches on device properties.
#### Examples of device properties and values to match on:

- **"type"** ("ethernet", "mdm", "wwan")
- **"tech"** ("3g", "lte", "5g", "lte/3g", "5g/lte")
- **"uid"** (modems, wwan, WAN mode VLAN UIDs)

#### Modems have additional properties to match on, including:
- **"port"** ("int1", "modem1")
- **"sim"** ("1" or "2")
- **"pdn"** ("1" or "2")

#### The above trigger_string example will match WAN devices of **type** "mdm" and **tech** "lte/3g".

### When the router detects a WAN device (both during boot and normal operation) it compares the properties of the device to all **trigger_string** criteria in WAN rules.  When it finds a match, it applies the properties of the WAN rule to the device.

## **"priority"**

#### The **priority** of a WAN rule is a float type variable that determines the devices order (priority) in the Connection Manager.  
#### The **LOWER** the value, the **HIGHER** priority.  
> It even supports negative numbers if you really want to be sure you take priority (e.g. -10)

## Additional values of WAN rules can apply additional properties to WAN devices.
#### Examples:
- **"def_conn_state"** Default Connection State ("normal", "standby", "alwayson")
- **"ip_mode"** IP Mode ("static", "dhcp", "pppoe")
- **"failure_check"** WAN Verify
- **"profiles"** APN Profiles (use with "selected_profile")

#### ... plus much more that can be found in the DTD.

---

# WAN Status

## WAN Status is found in `status/wan` 

### Two important fields in status/wan are:

- #### `status/wan/connection_state` - This is the state of the routers connection to the internet.  If any WAN devices are "up" this value will be "connected" otherwise it will be "disconnected"

- #### `status/wan/primary_device` - This is the current active WAN device(s).  If there are multiple devices it will be a list.

### Get and print your routers primary WAN device.  

---
```python
print(cp.get('status/wan/primary_device'))
```
---

#### Your turn:

### This is the **UID** of the current active WAN interface.

### Unique Identifiers ("UID") are defined as type **string** in the NCOS DTD.

### UIDs are used by NCOS to identify network devices (VLANs, modems, and Wifi-as-WAN Profiles).

> #### Example: WAN Device UIDs "ethernet-wan" and "mdm-1a2b3c4d"

<br>

## Device specific status is in `status/wan/devices/`

### `status/wan/devices` is a dictionary of WAN Devices and contains detailed status for each WAN device, including connection_state, signal strength, IP addresses, and more.

> #### This is where the Connection Manager pulls its info for displaying the staus of live devices.

### `status/wan/devices` lists WAN devices by their "UID" plus a prefix of the "type" of device:

- #### **ethernet** - The default WAN mode VLAN has a UID of simply "wan".  Ethernet ports have a "type" of "ethernet".  So the device shows up as "ethernet-wan".  The full path is `status/wan/devices/ethernet-wan`.

- #### **mdm** - Modems are of type "mdm" and NCOS generates dynamic UIDs (8 character hexadecimal).  An example modem device is "mdm-1a2b3c4d".  The full path being `status/wan/devices/mdm-1a2b3c4d`.

- #### **wwan** - Wifi-as-WAN are of type "wwan" and uses the SSID as part of the UID.  They also include a suffix to indicate the WiFi band ("2_4G" or "5G") plus a hypen (-) plus the radio number.  An example wwan device is "wwan-my_ssid:2_4G-1".  The full path being `status/wan/devices/wwan-my_ssid:2_4G-1`.

## List your WAN devices
> #### **PROTIP:** There is a lot of status information for each WAN device.  If you get and print the output of `status/wan/devices` you will get a huge wall of text!  In Python you can put a "\*" before a dictionary to refer to the string names of the dictionary keys (this is called "starred expression").  This is like the equivalent of doing "ls" instead of "get" at the CLI.  This is much easier to read than the object format of .keys().  Try it here:

---
```python
print(*cp.get('status/wan/devices'))
```
---

#### Your turn:

## Individual Device Status

### Get the UID of the primary device

### Then get the status of just the primary device by referring to the UID in the path:  

### `status/wan/devices/{UID}`  

> #### **PROTIP:** Use a python "f-string" to easily insert variables into text.  
> #### Simply put an **f** before the first quote of your string value (no space).  
> #### Then you can insert variables in curly braces right into your text.
> #### See the example on the second line below:

### Store the response in a variable and pretty-print it.

---
```python
uid = cp.get('status/wan/primary_device')
wan_status = cp.get(f'status/wan/devices/{uid}')
print(json.dumps(wan_status, indent=4))
```
---

#### Your turn:

## Get the config (WAN Rule) that matched and applied to this WAN device

### `status/wan/devices/{UID}/config`  

### Assign it to a variable and pretty-print it.

---
```python
wan_config = cp.get(f'status/wan/devices/{uid}/config')
print(json.dumps(wan_config, indent=4))
```
---

#### Your turn:

### You can see the WAN rule, including its "\_id\_" and config.

### Assign the rule "\_id\_" to a variable and print it.

---
```python
rule_id = wan_config["_id_"]
print(rule_id)
```
---

#### Your turn:

### You can use this rule_id ("\_id\_") to make config changes in `config/wan/rules2/{rule_id}`


# Additional CSClient() Methods:

# Using the **.post()** Method

## The .post() method creates a new item in a list/array in the NCOS API.

### Usage: `cp.post(path, json)`

<br>

## Creating a new VLAN using the **.post()** method...

### First you must define the VLAN you want to create:

#### Use the **REST API** to reference the DTD for VLANs (/api/dtd/config/vlan) to see the data types and which fields are required.  
#### The only field with "allowBlank=True" is "mac".  This means you must include all other fields (vid, uid, mode, ports).

#### REST API:

    {
		"type": "array",
		"nodes": {
			"vid": {
				"type": "u16",
				"min": 1,
				"max": 4094
			},
			"uid": {
				"type": "string"
			},
			"mode": {
				"type": "select",
				"optiontype": "string",
				"options": [
					["lan", "LAN"],
					["wan", "WAN"]
				]
			},
			"ports": {
				"type": "array",
				"nodes": {
					"_id_": {
						"type": "uuid"
					},
					"port": {
						"type": "u16"
					},
					"mode": {
						"type": "select",
						"optiontype": "string",
						"options": [
							["tagged", "Tagged"],
							["untagged", "Untagged"]
						]
					}
				}
			},
			"mac": {
				"type": "mac_address",
				"allowBlank": true
			}
		}

#### "vid" is of type **u16**, which is an "unsigned 16-bit integer", or just integer in Python.  
#### "uid" is of type string.
#### "mode" is of type **select** meaning it must be selected from available options ("lan" or "wan")
#### "ports" is an array that has nodes (fields) "port" and "mode".  For this example you won't use any ports, just an empty array.  An empty array is not a blank value!

## Create a JSON dictionary of the required fields to define a new VLAN.  

#### Pay close attention to your formatting!
> www.jsonlint.org is a great tool for validating JSON before putting it into code.

---
```python
vlan = {
    "vid": 10,  # 10 without quotes is an integer
    "uid": "New_VLAN",  # string
    "mode": "lan",  # REST API shows all options
    "ports": []  # Empty array = no ports
}
```
---

#### Your turn (you don't need to include the comments):

### Create your new VLAN using the **cp.post()** method.
### Assign the response to a variable for later use and print it:

---
```python
resp = cp.post('config/vlan', vlan)
print(resp)
```
---

#### If successful you will see a JSON response with `{'status': 'ok', 'data': listIndex}`

#### Your turn:

### Great work, you've now created a new VLAN!

#### Now let's delete it! :)

# Using the **.delete()** Method

## The .delete() method deletes the value at path (single value or entire object)

### Usage: `cp.delete(path)`

<br>

## Delete the previously created "New_VLAN" VLAN using the **.delete()** method...

#### If you recall, `config/vlans` is a list and uses UIDs, so we have to delete by index (not "\_id\_").  

### When you previously created the VLAN, the response included the list index as the value of the "data" key.  Reference that value to delete the VLAN, and print the response:

Watch your syntax and quotes here!

---
```python
print(cp.delete(f'config/vlan/{resp["data"]}'))
```
---

Successful response: `{'status': 'ok', 'data': 'True'}`

#### Your turn:


## What if you didn't know the list index?  How do you determine it?

#### Use a python [**for**](https://docs.python.org/3/tutorial/controlflow.html) loop to iterate through an [**enumerated**](https://docs.python.org/3/library/functions.html#enumerate) list of indexes and items (VLANs).
#### Match on the UID and use the index:

---
```python
vlans = cp.get('config/vlan')
for index, vlan in enumerate(vlans):
    if vlan["uid"] == "New_VLAN":
        print(cp.delete(f'config/vlan/{index}'))
```
---

> You'll need to recreate the VLAN since you deleted it in the last step

#### Your turn:


# Using the **.decrypt()** Method

## The .decrypt() method decrypts a value in NCOS with a "\$1\$" prefix.  It works like .get() for encrypted fields.

### Usage: `cp.decrypt(path)`

<br>

> `config/users` have passwords encypted with "\$2\$" prefixes and **can not** be unencrypted.

## Set the WPAPSK (wifi password) for the first BSS on the first wifi radio - it takes clear text and will encrypt it.
## Then get it and print it.
## Then decrypt and print it.

---
```python
cp.put('config/wlan/radio/0/bss/0/wpapsk', 'supersecret!')
print(cp.get('config/wlan/radio/0/bss/0/wpapsk'))
print(cp.decrypt('config/wlan/radio/0/bss/0/wpapsk'))
```
---

#### Your turn:

In [ ]:
cp.put('config/wlan/radio/0/bss/0/wpapsk', 'supersecret!')
print(cp.get('config/wlan/radio/0/bss/0/wpapsk'))
print(cp.decrypt('config/wlan/radio/0/bss/0/wpapsk'))

#### What's the difference?

# Using the **.patch()** Method

## The .patch() method merges a config into the router configuration.  Useful when needing to change multiple values at once.

### Usage: `cp.patch(config)`

<br>

> **config** must be a full config format like NCM: [ { ADD/CHANGE }, [ REMOVE ] ]


## Use .patch() to change the IP Address and netmask of the Guest LAN (1):

---
```python
new_config = [{
        "lan": {
            "1": {
                "ip_address": "192.168.11.1",
                "netmask": "255.255.255.128"
            }
        }
    },
    []
]
print(cp.patch(new_config))
```
---

#### Your turn:

### Now get and print the Guest LAN config to verify your changes:

# Using the Control Tree

# `control/`

## The control tree is used to issue commands to the router via NCOS API and does not effect the configuration.

## GPIOs are binary - they are either on or off, 1 or 0.

### Examples of control/ endpoints:
- #### **gpio** - turn on and off GPIOs (including LEDs)
- #### **firewall** - flush state tracking, flush ARP cache
- #### **ping** - send pings

### Get and print `control/gpio` to see available GPIOs:

## Use the **.put()** method to change a GPIO.

> #### LEDs are a great way to test because you can see the light turn on/off.
> #### You can even make some cool light shows with the E-series light bar.

---
```python
cp.put('control/gpio/LED_LOGO', 0)
```
---

#### Your turn:

# Congratulations! 

## You have completed Part 2!

## Find Part 3 in the file browser to the left!